# logistic regression

logistic regression是用回归变量x对logits进行线性回归

$$logit(p) = wx $$
$$logit(p) = log(\frac{p}{1-p}) = log(odds)$$
$$ p = logit^{-1}(wx)
    = sigmoid(wx) 
    = \frac{e^{wx}}{1+e^{wx}}$$

logit函数把在[0,1]范围内的变量转换到[$-\infty,+\infty$]  
sigmoid函数把[$-\infty,+\infty$]转换为概率  
sigmoid相加不等于1， 因为这是两个样本属于某类的概率相加，可能两个样本都很大该概率属于1，则相加接近于2

### 参数估计
- 最大似然估计
- 梯度上升

In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.estimator_checks import check_estimator
from sklearn.utils.multiclass import unique_labels
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.metrics import accuracy_score
#from Chemometrics.basic import sigmoid
from warnings import warn
import numpy as np

In [23]:
class LogisticRegression(BaseEstimator, ClassifierMixin):
    def __init__(self, lr=.1, seed=None, epsilon=1e-4):
        self.lr = lr
        self.seed = seed
        self.epsilon = epsilon
        self.fitted_ = False
    
    def fit(self, X, y=None):
        X, y = check_X_y(X, y)
        X = np.asarray(X, dtype=np.float32)
        np.random.seed(self.seed)
        self.classes_, y = np.unique(y, return_inverse=True)
        if not self.fitted_:
            self.w_ = np.random.randn(X.shape[1])
        if self.w_.shape[0] != X.shape[1]:
            self.w_ = np.random.randn(X.shape[1])
            
        N_run = 0
        while True:
            _grad_w = X.T.dot(y - sigmoid(X.dot(self.w_)))
            if np.linalg.norm(_grad_w) <= self.epsilon:
                break
            #todo: line search
            #newton
            self.w_ += self.lr*_grad_w
            N_run += 1
            if N_run > 1e5:
                warn(f'maximum steps reached, norm of gradient is {np.linalg.norm(_grad_w)}')
                break
        self.fitted_ = True
        return self
    
    @property
    def w(self):
        if not self.fitted_:
            warn('Not fitted')
        return self.w_

    def predict_proba(self, X, y=None):
        X = check_array(X)
        _score = X.dot(self.w)
        prob = sigmoid(_score)
        return prob
    
    def predict(self, X, y=None):
        X = check_array(X)
        prob = self.predict_proba(X, y)
        if len(self.classes_) == 1:
            return self.classes_[np.zeros_like(prob, dtype=np.int32)]
        yhat = self.classes_[np.argmax(np.vstack([1-prob, prob]).T, axis=1)]
        return yhat

    def score(self, X, y=None):
        return accuracy_score(y, self.predict(X))
def sigmoid(x):
    _exp = np.where(x > 0, np.exp(-x), np.exp(x))
    return np.where(x > 0, 1./(1. + _exp), _exp/(_exp + 1.))

In [24]:
# check_estimator(LogisticRegression)
# todo: 多分类

In [25]:
X = np.random.randn(5,20)
w = np.random.randn(20)
y = np.where(sigmoid(X.dot(w))>.5, 1, 0)
y
lr = LogisticRegression()

In [26]:
lr.fit(X, y)

LogisticRegression(epsilon=0.0001, lr=0.1, seed=None)

In [27]:
lr.predict(X), y

(array([0, 1, 1, 0, 0]), array([0, 1, 1, 0, 0]))

# 参考文献
[1] [Logit](https://en.wikipedia.org/wiki/Logit)  
[2] [Logistic Regression for Machine Learning](https://machinelearningmastery.com/logistic-regression-for-machine-learning/)  
[3] [sklearn.linear_model.LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)  
[4] [creating-your-own-estimator-scikit-learn](http://danielhnyk.cz/creating-your-own-estimator-scikit-learn/#get_paramsandset_params)  
[5] [Rolling your own estimator (scikit-learn docs)](http://scikit-learn.org/dev/developers/contributing.html#rolling-your-own-estimator)  
